In [1]:
import os
os.chdir('..')

from puzzle_generator import *

In [2]:
width = 6
height = 5
puzzle = RectangularPuzzle(
    width=width,
    height=height
)

puzzle.solving_timeout = 90

puzzle.naming = {
    None: '_',
}
puzzle.latex_naming = {
    None: ' ',
    'e': ' ',
    'square_sum': '{\\scalebox{0.7}{$\\square$}}',
    'big_square_sum': '{\\scalebox{1.0}{$\\square$}}',
}
for i in range(0, 10):
    puzzle.latex_naming[str(i)] = f'\\Large \\ensuremath{{{i}}}'

puzzle.domain_program += """
    num(0..9).
    value(N) :- num(N).
    #const cell_avg=4.
    guessed_number(square_sum,4*cell_avg).
    guessed_number(big_square_sum,9*cell_avg).
"""

puzzle.puzzle_gen_program += """
    { puzzle(C,V) : cell(C), value(V) }.
"""
puzzle.puzzle_constraints_program += """
    #minimize { 1,puzzle(C,V) : puzzle(C,V) }.
    
    :- square_cell(C1), 3 { puzzle(C2,V) : square_rel(C1,C2) } 3.
    
    :- square_cell(C1), guessed_number(square_sum,S),
        #sum { V,puzzle(C2,V) : square_rel(C1,C2), puzzle(C2,V) } = S.
"""

puzzle.solution_program = """
    1 { solution(C,V) : value(V) } 1 :- cell(C).
    solution(C,V) :- puzzle(C,V).
"""

puzzle.solution_program += """
    square_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height,
        Col < board_width.
    square_rel(c(Row,Col),c(Row,Col)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row,Col+1)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row+1,Col)) :- square_cell(c(Row,Col)).
    square_rel(c(Row,Col),c(Row+1,Col+1)) :- square_cell(c(Row,Col)).
    square_check(C1) :- square_cell(C1),
        guessed_number(square_sum,S),
        S = #sum { V2,C2 : solution(C2,V2), square_rel(C1,C2) }.
    :- square_cell(C), not square_check(C).
"""
puzzle.essential_solution_constraints = [
    """
    rows_diff(Row1,Row2) :- row(Row1), row(Row2), Row1 < Row2,
        col(Col), solution(c(Row1,Col),V1), solution(c(Row2,Col),V2),
        V1 != V2.
    :- row(Row1), row(Row2), Row1 < Row2, not rows_diff(Row1,Row2).

    cols_diff(Col1,Col2) :- col(Col1), col(Col2), Col1 < Col2,
        row(Row), solution(c(Row,Col1),V1), solution(c(Row,Col2),V2),
        V1 != V2.
    :- col(Col1), col(Col2), Col1 < Col2, not cols_diff(Col1,Col2).
    """,
    """
    big_square_cell(c(Row,Col)) :- cell(c(Row,Col)),
        Row < board_height-1,
        Col < board_width-1.
    big_square_rel(c(Row1,Col1),c(Row2,Col2)) :-
        big_square_cell(c(Row1,Col1)), cell(c(Row2,Col2)),
        Row1 <= Row2, Row2 <= Row1+2,
        Col1 <= Col2, Col2 <= Col1+2.
    big_square_check(C1) :- big_square_cell(C1),
        guessed_number(big_square_sum,S),
        S = #sum { V2,C2 : solution(C2,V2), big_square_rel(C1,C2) }.
    :- big_square_cell(C), not big_square_check(C).
    """,
]

In [3]:
puzzle.generate(
    verbose=True,
    precompute_solution=False,
)

Done grounding..
..
Stopped after solving timeout..
Solving time: 99.49 seconds



In [4]:
print(puzzle.pretty_repr_puzzle())

|_|_|2|_|_|6|
|_|_|_|5|_|_|
|_|_|_|_|_|_|
|_|_|_|_|_|_|
|_|_|_|_|_|_|

square_sum = 16
big_square_sum = 36


In [5]:
print(puzzle.pretty_repr_solution())

|1|7|2|5|3|6|
|5|3|4|5|3|4|
|5|3|6|1|7|2|
|3|5|2|7|1|6|
|3|5|4|3|5|4|


In [6]:
print(puzzle.latex_repr_puzzle())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) { };
      \node[anchor=center] (1;2) at (1.5,4.5) { };
      \node[anchor=center] (1;3) at (2.5,4.5) {\Large \ensuremath{2}};
      \node[anchor=center] (1;4) at (3.5,4.5) { };
      \node[anchor=center] (1;5) at (4.5,4.5) { };
      \node[anchor=center] (1;6) at (5.5,4.5) {\Large \ensuremath{6}};
      \node[anchor=center] (2;1) at (0.5,3.5) { };
      \node[anchor=center] (2;2) at (1.5,3.5) { };
      \node[anchor=center] (2;3) at (2.5,3.5) { };
      \node[anchor=center] (2;4) at (3.5,3.5) {\Large \ensuremath{5}};
      \node[anchor=center] (2;5) at (4.5,3.5) { };
      \node[anchor=center] (2;6) at (5.5,3.5) { };
      \node[anchor=center] (3;1) at (0.5,2.5) { };
      \node[anchor=center] (3;2) at (1.5,2.5) { };
      \node[anchor=center] (3;3) at (2.5,2.5) { };
      \node[anchor=center] (3;4) at (3.5,2.5) { }

In [7]:
print(puzzle.latex_repr_solution())


\begin{tikzpicture}[scale=1]
  \begin{scope}
    \draw (0,0) grid (6,5);
    \draw[ultra thick]
      (0,0) rectangle (6,5);
      \node[anchor=center] (1;1) at (0.5,4.5) {\Large \ensuremath{1}};
      \node[anchor=center] (1;2) at (1.5,4.5) {\Large \ensuremath{7}};
      \node[anchor=center] (1;3) at (2.5,4.5) {\Large \ensuremath{2}};
      \node[anchor=center] (1;4) at (3.5,4.5) {\Large \ensuremath{5}};
      \node[anchor=center] (1;5) at (4.5,4.5) {\Large \ensuremath{3}};
      \node[anchor=center] (1;6) at (5.5,4.5) {\Large \ensuremath{6}};
      \node[anchor=center] (2;1) at (0.5,3.5) {\Large \ensuremath{5}};
      \node[anchor=center] (2;2) at (1.5,3.5) {\Large \ensuremath{3}};
      \node[anchor=center] (2;3) at (2.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=center] (2;4) at (3.5,3.5) {\Large \ensuremath{5}};
      \node[anchor=center] (2;5) at (4.5,3.5) {\Large \ensuremath{3}};
      \node[anchor=center] (2;6) at (5.5,3.5) {\Large \ensuremath{4}};
      \node[anchor=cen